In [1]:
from mmdet.apis import init_detector, inference_detector
import mmcv
import matplotlib.pyplot as plt
import cv2
import os
import json
from tqdm.notebook import tqdm
import numpy as np
import logging

In [1]:
!mv demo_vids/ ../../data/

# Detect and Track


### First, load the ocr results, model weights and the config file

In [3]:
### Path to OCR of the video
data = open('../../data/ocr_results/results/ocr_with_gameclockrunning/2018-11-28_Virginia_at_Maryland/2018-11-28_Virginia_at_Maryland_ocr.json')
data = json.load(data)

### Config and model weights path
config_file = 'configs/yolo/custom_yolov3_d53_mstrain-608_273e_coco.py'
checkpoint_file = 'work_dirs/yolov3_d53_mstrain-608_273e_coco/epoch_273.pth'

### Path to the video
video_path = '../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4'

# build the model from a config file and a checkpoint file
model = init_detector(config_file, checkpoint_file, device='cuda:0')

In [326]:
logging.basicConfig(level=logging.DEBUG)
### Comment if you want to DEBUG
logging.disable(logging.DEBUG)

### Load the video using MMCV
video = mmcv.VideoReader(video_path)

### Temp variables
en = 0
proc = 0
pbar = tqdm(total=video.frame_cnt)
has_pred = 0


### Create the tracker
tracker = Tracker()

### Start prediction
for frame in video:
    
    ### Check if it is game moment using OCR
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running']:
        en += 1
        continue
    
    
    ### Predict the image and update stats
    result = inference_detector(model, frame)
    if len(result[0]) != 0: has_pred+=1
        
    ### Update tracker results
    tracker.update(result)
    
    ### Update stats
    en += 1
    proc += 1
    if proc > 600: break  
    pbar.update(1)
    
    
pbar.close()

In [327]:
### Here, I check the number of collected detection and processed frames
print(len(tracker.current_tracks), proc)

(601, 601, 12.398399120853863)

In [333]:
### Here I run the function which search for missing detections intervals in tracker.current_tracks 
### and interpolates its values. The below threshold is pointing how many missed values to interpolate.
how_much_to_interpoate = 6
tracker.calc_missing_intervals(how_much_to_interpoate)

0

In [325]:
class Tracker():
    
    def __init__(self):
        
        self.current_tracks = []  ### List of all tracks 
        self.temp_tracks = [] ### List of temporary tracks

        self.temp_track_len = 3 ### How much tracks to collect before merging into main track
        self.thresh_distance = 13 ### Euclidean distance threshold
        self.avg_dist = []
   

    def convert_to_center(self, result):
        return np.array([int((result[2] + result[0])/2), int((result[3] + result[1])/2)])

    
    def update(self, result):
        
        ### Check if we have any detections
        if len(result[0])==0:  
            
            self.current_tracks.append(None)
            self.temp_tracks = []
            self.zero_res = result
            
        elif len(result[0])==1:
            self.single_res = result[0][0]
            
            logging.debug("SINGLE RES")

            ### If we don't have approved tracks or no good history
            if len(self.current_tracks)==0 or self.current_tracks[-1] is None:
                logging.debug("NONE TRACK CONTINUED")
                ### if we don't temporary tracks we update temp tracks or we lost the track, else we check its distance to previous temp track
                if len(self.temp_tracks)==0:
                    self.temp_tracks.append(result[0][0])
                    self.current_tracks.append(None)

                else:
                    curr_center = self.convert_to_center(result[0][0])
                    last_center = self.convert_to_center(self.temp_tracks[-1])
                    self.avg_dist.append(np.linalg.norm(curr_center  - last_center))
                    
                    if np.linalg.norm(curr_center  - last_center) <  self.thresh_distance:
                        logging.debug("DIST CRITERIA SATISFIED")
                        self.temp_tracks.append(result[0][0]) 
                        
                        if len(self.temp_tracks) < self.temp_track_len:
                            self.current_tracks.append(None)
                        else:
                            logging.debug("CURRENT_TRACK UPDATED-------------------")
                            self.current_tracks.append(None)
                            self.current_tracks[-len(self.temp_tracks):] = self.temp_tracks.copy()
                            self.temp_tracks = []

                    else:
                        self.current_tracks.append(None)
                        self.current_tracks[-(len(self.temp_tracks)+1):] = [None for _ in range(len(self.temp_tracks)+1)]
                        self.temp_tracks = []

            else:
#                 self.temp_tracks = []
                ### Now, we work with tracks that have history greater than 5
                logging.debug("CURRENT TRACK CONTINUED")
                curr_center = self.convert_to_center(result[0][0])
                last_canter = self.convert_to_center(self.current_tracks[-1])
                
                if np.linalg.norm(curr_center - last_canter) <  self.thresh_distance:
                    logging.debug("GOOD TRACK APPENDED")
                    self.avg_dist.append(np.linalg.norm(curr_center - last_canter))
                    self.current_tracks.append(result[0][0])           
                else:
                    logging.debug("BAD TRACK APPENDED")
                    self.current_tracks.append(None)

        elif len(result[0])>1:
            self.multi_res = result
            
            logging.debug("MULTI RES")
#             self.temp_tracks = []
            
            if len(self.current_tracks)==0 or self.current_tracks[-1] is None:
                
                if len(self.temp_tracks)>0:
                    appended = False
                    
                    for i in range(len(result[0])):

                        curr_res = result[0][i]
                        curr_center = self.convert_to_center(curr_res)
                        last_center = self.convert_to_center(self.temp_tracks[-1])
                        if np.linalg.norm(curr_center - last_center) <  self.thresh_distance:
                                                                           
                            self.temp_tracks.append(result[0][0]) 
                            if len(self.temp_tracks) < self.temp_track_len:
                                self.current_tracks.append(None)
                            else:
                                logging.debug("CURRENT_TRACK UPDATED-------------------")
                                self.current_tracks.append(None)
                                self.current_tracks[-len(self.temp_tracks):] = self.temp_tracks.copy()
                                self.temp_tracks = []                         
                            
                            appended = True                       
                            break
                            
                    if not appended:
                        self.current_tracks.append(None)
                        self.current_tracks[-(len(self.temp_tracks)+1):] = [None for _ in range(len(self.temp_tracks)+1)]
                        self.temp_tracks = []    
                        
                else:
                    self.current_tracks.append(None)
            
            else:
#                 self.current_tracks.append(None)
                self.temp_tracks = []
                appended = False
                for i in range(len(result[0])):

                    curr_res = result[0][i]
                    curr_center = self.convert_to_center(curr_res)
                    last_center = self.convert_to_center(self.current_tracks[-1])
                    if np.linalg.norm(curr_center - last_center) <  self.thresh_distance:
                        appended = True
                        self.current_tracks.append(curr_res)                         
                        break

                if not appended:
                    self.current_tracks.append(None)
                    
                    
    def calc_missing_intervals(self, length=2):
    
        i=0
        j=0
        misses = 0
        while i<len(self.current_tracks) and j<len(self.current_tracks):
            if self.current_tracks[i] is not None:
                i += 1
                j = i       
            elif self.current_tracks[i] is None and self.current_tracks[j] is None:
                j += 1
            elif self.current_tracks[i] is None and self.current_tracks[j] is not None and 0 < (j - i) <= length:
                interp_tracks = self.interpolate_bboxes(self.current_tracks[i-1:j+1])
                self.current_tracks[i-1:j+1] = interp_tracks                
                i = j           
                misses += 1           
            elif self.current_tracks[i] is None and self.current_tracks[j] is not None and (j - i) > length:
                i=j


        if 0 < (j - i) <= length:
            misses += 1

        return misses
    
    
    def interpolate_bboxes(self, tracks):
        first = tracks[0]
        last = tracks[-1]

        first_center = [ (first[2] + first[0]) / 2, (first[3] + first[1]) / 2]
        last_center = [(last[2] + last[0]) / 2, (last[3] + last[1]) / 2]

        half_width = ((abs(first[2] - first[0]) + abs(last[2] - last[0])) / 2) / 2
        half_height = ((abs(first[3] - first[1]) + abs(last[3] - last[1])) / 2) / 2


        i = 1

        while i < len(tracks)-1:
            new_x = first_center[0] + ((i + 1) / len(tracks)) * (last_center[0] - first_center[0])
            new_y = first_center[1] + ((i + 1) / len(tracks)) * (last_center[1] - first_center[1])

            new_bbox = np.array([new_x - half_width, new_y - half_height, new_x + half_width, new_y + half_height, first[-1]])

            tracks[i] = new_bbox

            i+=1


        return tracks

# Visualize after tracker results

### After we process the video, we have all the processed detections in tracker.current_tracks (list of bboxes). The indecies of this list corresponds to index of the frame.

In [335]:
# test a video and show the results
video = mmcv.VideoReader('../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4')
out = cv2.VideoWriter('output_tracker.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, (1280,720))

en = 0
proc = 0
not_none = 0
for frame in video:
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running']:
        en += 1
        continue
    
    if tracker.current_tracks[proc] is not None:
        not_none += 1 
        result = np.expand_dims(tracker.current_tracks[proc][:4], axis=0)
        frame = mmcv.imshow_bboxes(frame, result, show=False, thickness=1, colors=['green'])

    
    out.write(frame)

    en += 1
    proc += 1
    if proc > 500:break
    
    
    
out.release()

In [92]:
video = mmcv.VideoReader('../../data/videos/videos/2018-11-28_Virginia_at_Maryland.mp4')

en = 0
proc = 0
not_none = 0
for frame in video:
    idx = str(en)
    if not data['results'][idx]['score_bug_present'] or not data['results'][idx]['game_clock_running'] or en<1000:
        en += 1
        continue
    
    if proc%5==0:
        cv2.imwrite('../../data/single_frames/frame_{:05}.jpg'.format(int(proc/5)), frame)
    proc += 1
    if proc>160:break

In [93]:
sorted(os.listdir('../../data/single_frames/'))

['frame_00000.jpg',
 'frame_00001.jpg',
 'frame_00002.jpg',
 'frame_00003.jpg',
 'frame_00004.jpg',
 'frame_00005.jpg',
 'frame_00006.jpg',
 'frame_00007.jpg',
 'frame_00008.jpg',
 'frame_00009.jpg',
 'frame_00010.jpg',
 'frame_00011.jpg',
 'frame_00012.jpg',
 'frame_00013.jpg',
 'frame_00014.jpg',
 'frame_00015.jpg',
 'frame_00016.jpg',
 'frame_00017.jpg',
 'frame_00018.jpg',
 'frame_00019.jpg',
 'frame_00020.jpg',
 'frame_00021.jpg',
 'frame_00022.jpg',
 'frame_00023.jpg',
 'frame_00024.jpg',
 'frame_00025.jpg',
 'frame_00026.jpg',
 'frame_00027.jpg',
 'frame_00028.jpg',
 'frame_00029.jpg',
 'frame_00030.jpg',
 'frame_00031.jpg',
 'frame_00032.jpg']

In [94]:
!zip -r frames.zip ../../data/single_frames/

  adding: ../../data/single_frames/ (stored 0%)
  adding: ../../data/single_frames/frame_00025.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00004.jpg (deflated 1%)
  adding: ../../data/single_frames/frame_00024.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00028.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00031.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00007.jpg (deflated 1%)
  adding: ../../data/single_frames/frame_00018.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00013.jpg (deflated 1%)
  adding: ../../data/single_frames/frame_00030.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00006.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00032.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00017.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00011.jpg (deflated 0%)
  adding: ../../data/single_frames/frame_00008.jpg (deflated 1%)
  adding: ../../data/single_frames/frame_0

In [91]:
!rm ../../data/single_frames/*
!rm frames.zip

In [26]:
!sudo apt install zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 26 not upgraded.
Need to get 167 kB of archives.
After this operation, 638 kB of additional disk space will be used.
Get:1 http://us-east-2.ec2.archive.ubuntu.com/ubuntu bionic/main amd64 zip amd64 3.0-11build1 [167 kB]
Fetched 167 kB in 0s (1426 kB/s)

78Selecting previously unselected package zip.
(Reading database ... 256411 files and directories currently installed.)
Preparing to unpack .../zip_3.0-11build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 17%] [#########.................................................] 8Unpacking zip (3.0-11build1) ...
7Progress: [ 33%] [###################.......................................] 87Progress: [ 50%] [#############################.............................] 8Setting up zi